In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import pipeline, AutoTokenizer, BitsAndBytesConfig
import torch
from tqdm import tqdm
import json
import numpy as np
from typing import List
import pandas as pd
import random
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

e:\Anaconda\envs\mnlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [28]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained("Tachi67/mnlp_dpo_model_bloom")
tokenizer = AutoTokenizer.from_pretrained("Tachi67/mnlp_dpo_model_bloom")

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.03it/s]


In [44]:
model.to(device)

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 2048)
    (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
  )
  (

In [94]:
original_model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-1b7")
original_model.to(device)

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 2048)
    (word_embeddings_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
  )
  (

In [6]:
from datasets import load_dataset
dataset = load_dataset("Tachi67/mnlp_dpo_data_7k")

Generating test split: 100%|██████████| 657/657 [00:00<00:00, 164252.11 examples/s]


In [160]:
def prep_input(input_sample:dict):
    prompt = input_sample['prompt']
    question_body = prompt.split('Options:')[0].strip()
    options = prompt.split('Options:')[1].strip()
    
    
    instruction = f"""
Answer the following question:
{question_body}
Choose from the following options:
Options:\n{options}
    
Please provide only one answer with the letter of the option.
The chosen option is:
    """

    return instruction, question_body, options

def generate_answer(instruction, model, tokenizer, max_new_tokens=10):
    input = tokenizer.encode(instruction, return_tensors="pt").to(device)
    output = model.generate(input, max_new_tokens=max_new_tokens)
    output_str = tokenizer.decode(output[0], skip_special_tokens=True)
    return output_str

In [ ]:
def parse_mcqa_output(s:str, instruction:str):
    s = s[len(instruction):]
    def find_alphabetical_answer(s):
        # 初始化最佳匹配结果为None
        best_index = len(s)  # 初始值设为字符串长度，比任何有效索引都大
        best_char = None
        characters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

        # 遍历每个字符
        for char in characters:
            index = s.find(char)
            if index != -1 and index < best_index:  # 检查是否找到并且是否是更早的索引
                best_index = index
                best_char = char

        # 如果没有找到任何字符，返回-1
        if best_char is None:
            raise ValueError("No alphabetical character found in the string")

        # 返回找到的最早字符的索引
        return best_index
    return s[find_alphabetical_answer(s)]

TODO:

考虑如下情况：

model的output是 
(a)，或者直接返回了content而不是label（ABCD..）
Option: A
The chosen option is:
     (a) True
     (b) False
没有回复


在原来的算法里，不会捕捉到A。

考虑：parse这样的情形；ask again；match content；从option中获取option的数量

In [171]:
index = random.randint(0, len(dataset['test']))
instruction, question_body, options = prep_input(dataset['test'][index])
print(instruction)
print("===================")

output_str_new_model = generate_answer(instruction, model, tokenizer)
print(output_str_new_model)
print("MCQA ANSWER ----------------->", parse_mcqa_output(output_str_new_model, instruction))

print("-------------------")


output_str_original_model = generate_answer(instruction, original_model, tokenizer)
print(output_str_original_model[len(input):])
print("MCQA ANSWER ----------------->", parse_mcqa_output(output_str_original_model, instruction))



Answer the following question:
Question: In JOS, after finishing the execution of a user-level page fault handler, how is the program control flow transferred back to the program? (You may get insights from the code snippet of _pagefault_upcall.)?
Choose from the following options:
Options:
A. The control flow will be transferred to kernel first, then to Env that caused the page fault.
B. The control flow will be transferred to Env that caused the page fault directly.
    
Please provide only one answer with the letter of the option.
The chosen option is:
    

Answer the following question:
Question: In JOS, after finishing the execution of a user-level page fault handler, how is the program control flow transferred back to the program? (You may get insights from the code snippet of _pagefault_upcall.)?
Choose from the following options:
Options:
A. The control flow will be transferred to kernel first, then to Env that caused the page fault.
B. The control flow will be transferred to